# Tiny Chronicler @( * O * )@ ~ Compositon Generator

Determine the parameters for the music and video for each "module" (which comes from the generated MIDI file).

This is a script for testing and development of the "Tiny Chronicler" and will be integrated into the actual program one day.

In [255]:
import array
import numpy as np

In [301]:
def to_dict(namespace, arr):
    vals = {}
    for index, key in enumerate(arr):
        vals[key] = key
    return vals
        

PARAMETERS = to_dict("parameters", [
    "PHOTO",
    "VIDEO_ONLY",
    "VIDEO_W_SOUND",
    "BLACK",
    "NARRATOR",
    "VOCODER",
    "VOCODER_RELEASE_0.34",
    "VOCODER_RELEASE_0.65",
    "VOCODER_REVERB_0",
    "VOCODER_REVERB_-11",
    "TINY_CHRONICLER",
    "CHOIR",
    "BASS",
    "KAJSA_M",
    "KAJSA_L",
    "MALTE",
    "VOICES_SHORT",
    "VOICES_LONG",
    "VOICES_HP_0.6",
    "VOICES_HP_1.0",
    "VOICES_REVERB_-23",
    "VOICES_REVERB_-6"
])

SCENES = {
    "ONLY_NARRATOR": [
        PARAMETERS["NARRATOR"],
    ],
    "NARRATOR_W_2VOICES_HP": [
        PARAMETERS["NARRATOR"],
        PARAMETERS["KAJSA_M"],
        PARAMETERS["MALTE"],
        PARAMETERS["VOICES_HP_0.6"],
        PARAMETERS["VOICES_SHORT"],
    ],
    "2VOICES": [
        PARAMETERS["KAJSA_M"],
        PARAMETERS["MALTE"],
        PARAMETERS["VOICES_HP_1.0"],
        PARAMETERS["VOICES_SHORT"],
    ],
}

MOVEMENTS = [
    {
        "name": "MOVEMENT_1",
        "percentage": 0.2,
        "sections": [
            {
                "name": "MOVEMENT_1_SECTION_1",
                "percentage": 0.15,
                "scenes": [
                    {
                        "name": "MOVEMENT_1_SECTION_1_SCENE_1",
                        "percentage": 1,
                        "parameters": SCENES["ONLY_NARRATOR"]
                    },
                ]
            },
            {
                "name": "MOVEMENT_1_SECTION_2",
                "percentage": 0.8,
                "scenes": [
                    {
                        "name": "MOVEMENT_1_SECTION_2_SCENE_1",
                        "percentage": 0.2,
                        "parameters": SCENES["ONLY_NARRATOR"]
                    },
                    {
                        "name": "MOVEMENT_1_SECTION_2_SCENE_2",
                        "percentage": 0.5,
                        "parameters": SCENES["NARRATOR_W_2VOICES_HP"]
                    },
                    {
                        "name": "MOVEMENT_1_SECTION_2_SCENE_3",
                        "percentage": 0.3,
                        "parameters": SCENES["2VOICES"]
                    }
                ]
            },
            {
                "name": "MOVEMENT_1_SECTION_3",
                "percentage": 0.05,
                "scenes": [
                    {
                        "name": "MOVEMENT_1_SECTION_3_SCENE_1",
                        "percentage": 1,
                        "parameters": SCENES["ONLY_NARRATOR"]
                    },
                ]
            },
        ],
    },
    {
        "name": "MOVEMENT_2",
        "percentage": 0.8,
        "sections": [
            {
                "name": "MOVEMENT_2_SECTION_1",
                "percentage": 1,
                "scenes": [
                    {
                        "name": "MOVEMENT_2_SECTION_1_SCENE_1",
                        "percentage": 1,
                        "parameters": SCENES["ONLY_NARRATOR"]
                    },
                ]
            },
        ],
    },
]

In [306]:
def check_movements(config):
    movement_total_percentage = 0
    
    for movement in config:
        movement_total_percentage += movement['percentage']
        sections_total_percentage = 0
        for section in movement['sections']:
            sections_total_percentage += section['percentage']
            scenes_total_percentage = 0
            for scene in section['scenes']:
                scenes_total_percentage += scene['percentage']
            if scenes_total_percentage != 1:
                raise Exception("Invalid scene total percentage (not 100%)")
        if sections_total_percentage != 1:
            raise Exception("Invalid sections total percentage (not 100%)")

    if movement_total_percentage != 1:
        raise Exception("Invalid movement total percentage (not 100%)")


class CompositionGenerator:
    
    def __init__(self, movements):
        check_movements(movements)
        self.movements = movements
        
    def score(self, modules):
        current_movement = 0
        current_section = 0
        
        movement_start = 0
        movement_end = self.movements[current_movement]['percentage']
        
        section_start = 0
        section_end = self.movements[current_movement]['sections'][current_section]['percentage']
        
        result = []

        for (index, module) in enumerate(modules):
            current_position = index / len(modules)

            # Determine movement
            if movement_end < current_position:
                current_movement += 1
                current_section = 0
                section_start = 0
                section_end = self.movements[current_movement]['sections'][current_section]['percentage']
                movement_start = movement_end
                movement_end = movement_start + self.movements[current_movement]['percentage']
            
            # Determine section
            current_section_position = (current_position - movement_start) / (movement_end - movement_start)
            if section_end < current_section_position:
                current_section += 1
                section_start = section_end
                section_end = section_start + self.movements[current_movement]['sections'][current_section]['percentage']
            
            scenes = self.movements[current_movement]['sections'][current_section]['scenes']
            scenes_probabilities = [s['percentage'] for s in scenes]
            
            # Determine scene
            scene = np.random.choice(scenes, None, True, scenes_probabilities)
                    
            result.append({
                "parameters": scene['parameters'],
                "module": module,
            })
            
        return result

In [317]:
# Create "fake" modules
modules = [0 for i in range(25)]

# Generate composition
generator = CompositionGenerator(MOVEMENTS)
score = generator.score(modules)
score

[{'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR',
   'KAJSA_M',
   'MALTE',
   'VOICES_HP_0.6',
   'VOICES_SHORT'],
  'module': 0},
 {'parameters': ['KAJSA_M', 'MALTE', 'VOICES_HP_1.0', 'VOICES_SHORT'],
  'module': 0},
 {'parameters': ['NARRATOR',
   'KAJSA_M',
   'MALTE',
   'VOICES_HP_0.6',
   'VOICES_SHORT'],
  'module': 0},
 {'parameters': ['NARRATOR',
   'KAJSA_M',
   'MALTE',
   'VOICES_HP_0.6',
   'VOICES_SHORT'],
  'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'parameters': ['NARRATOR'], 'module': 0},
 {'param